In [ ]:
import duckdb
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
import os

# load environment variables from .env file for project
dotenv_path = Path('../.env')
load_dotenv(dotenv_path=dotenv_path)

data_directory = os.getenv("OUTPUT_DIRECTORY")

In [ ]:
os.listdir(data_directory)

In [ ]:
duck_db_temp_dir = os.path.join(data_directory, "duck_db_temp")
if not os.path.exists(duck_db_temp_dir):
    os.makedirs(duck_db_temp_dir)

In [ ]:
con = duckdb.connect("ssi_plus.db", config={"temp_directory": duck_db_temp_dir})

In [ ]:
con.sql("select current_setting('temp_directory')")

In [ ]:
revenue_filename = os.path.join(data_directory, 'ssi_plus_revenue.parquet')
con.sql('drop table if exists plus_revenue')
con.sql(f"create table plus_revenue as select date_trunc('day', strptime(year_month, '%Y%m')) as start_date, last_day(strptime(year_month, '%Y%m')) as end_date, * from read_parquet('{revenue_filename}')")
con.sql("create index ean_idx on plus_revenue (ean_number)")

In [ ]:
df = pd.read_parquet(revenue_filename, engine="pyarrow")
df.columns

In [ ]:
df.head()

In [ ]:
con.sql("select * from plus_revenue limit 5")

In [ ]:
con.sql("select  date_trunc('day', strptime(year_month, '%Y%m')) as start_date, last_day(strptime(year_month, '%Y%m')) as end_date, * from plus_revenue limit 10")

In [ ]:
con.sql("select start_date, end_date, ean_number, ean_name from plus_revenue where start_date = '2022-03-01'")

In [ ]:
plus_output_filename = os.path.join(data_directory, 'OutputEansCoicopsPlus_202107_202205.parquet')

con.sql("drop table if exists plus_output")
con.sql(f"create table plus_output as select * from read_parquet('{plus_output_filename}')")
con.sql("create index po_rep_id on plus_output (rep_id)")
con.sql("create index po_ean_number on plus_output (ean_number)")
con.sql("select * from plus_output")

In [ ]:
con.sql("drop table if exists plus_receipts")

plus_receipt_filename = os.path.join(data_directory, 'KassabonPlus_va_202201_Prd.parquet')
con.sql(f"create table plus_receipts as select * from read_parquet('{plus_receipt_filename}')")
con.sql("create index pr_ean_idx on plus_receipts (ean_number)")
con.sql("create index pr_rep_id on plus_receipts (rep_id)")

In [ ]:
con.sql("select * from plus_receipts")

In [ ]:
con.sql("""select rep_id, count(distinct(receipt_text)) from plus_receipts
group by rep_id 
having count(distinct(receipt_text)) > 1
order by count(distinct(receipt_text))
""")

In [ ]:
con.sql(""" select count(*) from
(
select rep_id, count(distinct(receipt_text)) from plus_receipts
group by rep_id 
having count(distinct(receipt_text)) = 1
order by count(distinct(receipt_text))
)
""")

In [ ]:
con.sql(""" select count(*) from
(
select rep_id, count(distinct(receipt_text)) from plus_receipts
group by rep_id 
having count(distinct(receipt_text)) > 1
order by count(distinct(receipt_text))
)
""")

In [ ]:
con.sql(""" select count(distinct(rep_id)) from plus_receipts""")

In [ ]:
292489 + 443950 

In [ ]:
con.sql("""select count(*) from plus_receipts""")

In [ ]:
con.sql("""drop table if exists plus_receipt_ean;
create table plus_receipt_ean as 
select * from plus_revenue as pr
inner join plus_receipts as po on pr.rep_id = po.rep_id and po.start_date >= pr.start_date and po.start_date <= pr.end_date
""")

con.sql("select * from plus_receipt_ean")

In [ ]:
con.sql("select count(*) from plus_receipt_ean")

In [ ]:
con.sql("select count(*) from plus_receipts")

In [ ]:
con.sql("select count(*) from plus_revenue")

In [ ]:
con.sql("select count(*) from plus_revenue where start_date >= '2022-03-01' ")

In [ ]:
con.sql("""drop table if exists plus_receipt_ean_left;
create table plus_receipt_ean_left as
select * from plus_revenue as pr
left join plus_receipts as po on pr.rep_id = po.rep_id and po.start_date >= pr.start_date and po.start_date <= pr.end_date
""")

con.sql("select * from plus_receipt_ean_left")

In [ ]:
con.sql("select count(*) from plus_receipt_ean_left")

In [ ]:
con.sql("select count(distinct(rep_id)) from plus_receipt_ean_left")

In [ ]:
con.sql("select count(distinct(rep_id)) from plus_revenue")

In [ ]:
con.sql("select count(distinct(rep_id)) from plus_receipts")

In [ ]:
con.sql("""
select count(distinct(pre.rep_id)) from (
select distinct(rep_id) as rep_id from plus_revenue
) pre
left join ( 
    select distinct(rep_id) as rep_id from plus_receipts
) pr on pre.rep_id = pr.rep_id
where pr.rep_id is null
""")

In [ ]:
con.sql("select count(*) from plus_revenue")

In [ ]:
con.sql(""" select sum(rep_id_count) from 
(
    select rep_id, count(*) as rep_id_count from plus_revenue 
    where start_date >= '2022-03-01'
    group by rep_id 
    having count(rep_id) > 1
) rc
""")

In [ ]:
7119674 - 3874826 

In [ ]:
con.sql("select count(*) from plus_receipt_ean_left where receipt_text is null  and start_date >= '2022-03-01'")

In [ ]:
con.sql("select start_date, end_date, ean_name, rep_id from plus_receipt_ean_left where receipt_text is null and start_date >= '2022-03-01' ")

In [ ]:
con.sql("create index pre_rep_id_idx on plus_receipt_ean (rep_id)")

In [ ]:
plus_receipts_df = con.sql("""select date_part('year', start_date) as year, date_part('month', start_date) as month, count(distinct(rep_id)) from plus_receipts 
            group by date_part('year', start_date), date_part('month', start_date) 
            order by date_part('year', start_date), date_part('month', start_date)
            
            """).df()

In [ ]:
plus_receipts_df.head()

In [ ]:
plus_revenue_products_df = con.sql("""select date_part('year', start_date) as year, date_part('month', start_date) as month, count(distinct(rep_id)) from plus_revenue 
            group by date_part('year', start_date), date_part('month', start_date) 
            order by date_part('year', start_date), date_part('month', start_date)
            
            """).df()

In [ ]:
plus_revenue_products_df.head()

In [ ]:
product_counts_df = plus_receipts_df.merge(plus_revenue_products_df, on=["year", "month"])
product_counts_df["diff"] = product_counts_df["count(DISTINCT rep_id)_x"] - product_counts_df["count(DISTINCT rep_id)_y"]
product_counts_df

In [ ]:
con.sql("""select count(*) from plus_receipt_ean
        where receipt_text is null
        """)

In [ ]:
con.sql("select min(start_date), max(start_date) from plus_receipt_ean")

In [ ]:
con.sql("select min(start_date), max(start_date) from plus_receipts")

In [ ]:
# Hence all dates before 2022-03-28 are dropped because of the inner join
con.sql("select min(start_date), max(start_date) from plus_revenue")

In [ ]:
con.sql("""
drop view if exists double_coicops;
create table double_coicops as
select rep_id, count(distinct(coicop_number)), string_agg(coicop_number, ', ' order by start_date) as coicop_numbers, string_agg(receipt_text, ', ' order by receipt_text) as receipt_texts
from plus_receipt_ean
group by rep_id
having count(distinct(coicop_number)) > 1
order by rep_id
""")

In [ ]:
con.sql("select count(*) from  double_coicops")

In [ ]:
double_coicops_df = con.sql("select * from  double_coicops").df()

In [ ]:
len(double_coicops_df)

In [ ]:
double_coicops_df

In [ ]:
double_coicops_df.to_csv("double_coicops.csv", sep=";", index=False)

In [ ]:
plus_receipt_filename = os.path.join(data_directory, 'ssi_plus_revenue_receipts.parquet')

con.sql(f"copy plus_receipt_ean to '{plus_receipt_filename}' (format parquet);")

In [ ]:
# we do not have a lot of matches :(
con.sql("select count(*) from plus_receipts"), con.sql("select count(*) from plus_receipt_ean"), con.sql("select count(*) from plus_output")

In [ ]:
con.sql("select  from plus_receipt_ean")

In [ ]:
con.sql("select * from plus_receipts limit 5")

In [ ]:
con.sql("select * from plus_receipts where Datum_vanaf >= '2022-03-01' and Datum_vanaf <= '2022-03-31'")

In [ ]:
con.sql("select Kassabon from plus_receipts where Kassabon like 'PLUS%' order by Kassabon")

In [ ]:
con.sql("""select ean_name, Kassabon from plus_revenue as pr 
        inner join plus_receipts as pc on pr.ean_name = pc.Kassabon 
        order by ean_name""")

In [ ]:
con.sql("""select ean_name, Kassabon from ( 
            select distinct(ean_name) from plus_revenue
        ) as pr 
        inner join 
        (
            select distinct(Kassabon) from plus_receipts
        ) as pc on pr.ean_name = pc.Kassabon 
        order by ean_name""")

In [ ]:
con.sql("""select len(ean_name), count(*) from ( 
            select distinct(ean_name) from plus_revenue
        ) as pr 
        inner join 
        (
            select distinct(Kassabon) from plus_receipts
        ) as pc on pr.ean_name = pc.Kassabon 
        group by len(ean_name)
        order by len(ean_name)""").df().plot.bar(x="len(ean_name)", y="count_star()")

In [ ]:
con.sql("select count(distinct(ean_name)) from plus_revenue"), con.sql("select count(distinct(Kassabon)) from plus_receipts")

In [ ]:
con.sql("""select ean_name, Kassabon from ( 
            select distinct(ean_name) from plus_revenue
        ) as pr 
        left join 
        (
            select distinct(Kassabon) from plus_receipts
        ) as pc on pr.ean_name = pc.Kassabon 
        where Kassabon is null
        order by ean_name
        """)

In [ ]:
con.sql("""select ean_name, Kassabon from ( 
            select distinct(ean_name) from plus_revenue
        ) as pr 
        right join 
        (
            select distinct(Kassabon) from plus_receipts
        ) as pc on pr.ean_name = pc.Kassabon 
        where ean_name is null
        order by ean_name
        """)

In [ ]:
con.sql("""select count(*) from ( 
            select distinct(trim(ean_name)) as ean_name from plus_revenue
        ) as pr 
        right join 
        (
            select distinct(trim(Kassabon)) as Kassabon from plus_receipts
        ) as pc on pr.ean_name = pc.Kassabon 
        where ean_name is null
        """)

In [ ]:
con.sql("select len(ean_name), count(*) from plus_revenue group by len(ean_name) order by len(ean_name)")

In [ ]:
# Ean name lengths are larger than 20 characters
con.sql("select len(ean_name), count(*) from plus_revenue group by len(ean_name) order by len(ean_name)").df().plot.bar(x="len(ean_name)", y="count_star()", figsize=(10,10))

In [ ]:
con.sql("select len(Kassabon), count(*) from plus_receipts group by len(Kassabon) order by len(Kassabon)")

In [ ]:
# Maximum length of kassabon for Plus is 20 characters (see mail Sigrid)
con.sql("select len(Kassabon), count(*) from plus_receipts group by len(Kassabon) order by len(Kassabon)").df().plot.bar(x="len(Kassabon)", y="count_star()")

In [ ]:
con.sql("select * from plus_receipts where len(Kassabon) > 20")

In [ ]:
con.sql("""select count(*) from ( 
            select distinct(trim(ean_name)) as ean_name from plus_revenue
        ) as pr 
        left join 
        (
            select distinct(trim(Kassabon)) as Kassabon from plus_receipts
        ) as pc on pr.ean_name = pc.Kassabon 
        where Kassabon is null
        """)

In [ ]:
con.sql("""select count(*) from plus_revenue as pr 
        inner join plus_receipts as pc on pr.ean_name = pc.Kassabon 
       """), con.sql("select count(*) from plus_receipts"), con.sql("select count(*) from plus_revenue")

In [ ]:
con.sql("select min(Datum_vanaf) from plus_receipts rode")

In [ ]:
con.sql("""create table plus_receipt_revenue as 
select pr.*, pc.kassabon from plus_revenue as pr 
inner join plus_receipts as pc on pr.ean_number = pc.ean_number 
where pc.Datum_vanaf >= pr.start_date and pc.Datum_vanaf <= pr.end_date
""")

In [ ]:
con.sql("select * from plus_receipt_revenue")

In [ ]:
con.sql("select ean_name, Kassabon from plus_receipt_revenue order by ean_name")

In [ ]:
con.sql("COPY plus_receipt_revenue TO 'plus_receipt_revenue.parquet' (FORMAT PARQUET);")

In [ ]:
con.sql("select count(*) from plus_receipt_revenue")

In [ ]:
con.sql("select count(*) from plus_receipts")

In [ ]:
con.sql("select count(*) from plus_revenue")